In [1]:
# Import Libraries
import numpy as np
import json

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

In [21]:
#Upload VGG16
# plus besoin car déja chargé
#vgg16 = VGG16(weights='imagenet')

In [22]:
# plus besoin car déja chargé
#vgg16.save('vgg16.h5')

In [3]:
import tensorflow as tf
vgg16 = tf.keras.models.load_model('vgg16.h5')

In [4]:
def prepare_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [5]:
y = prepare_image('/home/liganium/Documents/Django/Django App- Create & Deploy a Computer Vision Model/images/image1.jpg')

In [6]:
preds = vgg16.predict(y)

In [7]:
preds.shape

(1, 1000)

In [8]:
preds

array([[2.23652474e-08, 1.43624845e-07, 5.68609586e-08, 1.54170525e-08,
        7.67949970e-08, 1.62230780e-08, 6.92965907e-09, 1.21872263e-05,
        2.16029321e-06, 1.77866227e-07, 6.91911097e-08, 1.07023219e-07,
        4.47992278e-07, 4.66504453e-08, 2.31282034e-08, 1.33918675e-07,
        3.42524856e-07, 1.08745802e-07, 4.47940153e-07, 3.64491740e-07,
        1.05713326e-07, 2.09610434e-07, 7.48158129e-07, 9.94437323e-06,
        3.03227097e-07, 6.90029992e-07, 3.14322108e-07, 2.80006304e-07,
        2.42353423e-07, 4.01876719e-08, 1.53816842e-07, 3.91484008e-07,
        2.22585896e-07, 3.57389666e-07, 5.32762760e-06, 1.78308744e-06,
        1.32309685e-06, 1.68673694e-06, 3.39059056e-07, 1.73098968e-07,
        6.98893359e-07, 1.03933905e-06, 1.24702012e-06, 1.31120970e-07,
        9.72953558e-07, 2.36412831e-07, 2.38911895e-07, 3.69429145e-08,
        2.12779781e-07, 2.62668806e-07, 2.33518193e-07, 8.70960051e-08,
        2.52551081e-06, 2.23427023e-07, 3.55203468e-07, 1.089304

In [9]:
CLASS_INDEX = None

In [10]:
CLASS_INDEX_PATH = 'imagenet_class_index.json'
#CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'

In [11]:
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    #Load the jason file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [12]:
print (get_predictions(preds, top=5))

[[('n02814533', 'beach_wagon', 0.42408755), ('n04285008', 'sports_car', 0.25471827), ('n03459775', 'grille', 0.10582006), ('n02974003', 'car_wheel', 0.09170451), ('n03100240', 'convertible', 0.048233032)]]


In [ ]:
#First Check - category list creation

In [13]:
from collections import Counter, defaultdict
import os
import pickle as pk

In [16]:
def get_car_categories():
    d = defaultdict(float)
    img_list = os.listdir('01-whole')
    for i, img_path in enumerate(img_list):
        img = prepare_image('01-whole/'+img_path)
        out = vgg16.predict(img)
        top = get_predictions(out, top=5)
        for j in top[0]:
            d[j[0:2]] += j[2]
        if i % 100 == 0:
            print (i, '/', len(img_list), 'complete')
    return Counter(d)

In [17]:
cat_counter=get_car_categories()

0 / 920 complete
100 / 920 complete
200 / 920 complete
300 / 920 complete
400 / 920 complete
500 / 920 complete
600 / 920 complete
700 / 920 complete
800 / 920 complete
900 / 920 complete


In [19]:
cat_counter

Counter({('n02814533', 'beach_wagon'): 67.81617866217857,
         ('n03770679', 'minivan'): 132.04886387911392,
         ('n03100240', 'convertible'): 91.41381258505862,
         ('n04285008', 'sports_car'): 116.25077116955072,
         ('n03769881', 'minibus'): 20.396584746966255,
         ('n02930766', 'cab'): 17.854767971206456,
         ('n03930630', 'pickup'): 65.0029500547098,
         ('n02974003', 'car_wheel'): 17.176701675503864,
         ('n04037443', 'racer'): 39.565461611549836,
         ('n03459775', 'grille'): 18.050604918651516,
         ('n04461696', 'tow_truck'): 18.77435179578606,
         ('n04065272', 'recreational_vehicle'): 3.719219581456855,
         ('n03445924', 'golfcart'): 2.6788842319510877,
         ('n03796401', 'moving_van'): 14.7552862483426,
         ('n03670208', 'limousine'): 53.60916478239233,
         ('n04467665', 'trailer_truck'): 3.239227047888562,
         ('n03594945', 'jeep'): 30.023130393470638,
         ('n04252225', 'snowplow'): 7.08604650

In [21]:
cat_counter.most_common()[0:8]

[(('n03770679', 'minivan'), 132.04886387911392),
 (('n04285008', 'sports_car'), 116.25077116955072),
 (('n03100240', 'convertible'), 91.41381258505862),
 (('n02814533', 'beach_wagon'), 67.81617866217857),
 (('n03930630', 'pickup'), 65.0029500547098),
 (('n03670208', 'limousine'), 53.60916478239233),
 (('n04037443', 'racer'), 39.565461611549836),
 (('n03594945', 'jeep'), 30.023130393470638)]

In [18]:
#with open('cat_counter.pk', 'wb') as f:
#    pk.dump(cat_counter,f,-1)

In [42]:
# This piece of code will be used to load already saved cat_counter
#with open('cat_counter.pk', 'rb') as f:
 #   cat_counter = pk.load(f)

In [24]:
cat_list  = [k for k, v in cat_counter.most_common()[:27]]

In [25]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n02974003', 'car_wheel'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03445924', 'golfcart'),
 ('n03345487', 'fire_engine'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian'),
 ('n03417042', 'garbage_truck'),
 ('n02965783', 'car_mirror')]

# First Check - car or not

In [26]:
def car_categories_gate(image_path, cat_list):
    img = prepare_image(image_path)
    out = vgg16.predict(img)
    top = get_predictions(out, top=5)
    print ("Validating that this is a picture of your car...")
    for j in top[0]:
        if j[0:2] in cat_list:
            print (j[0:2])
            return "Validation complete - proceed to damage evaluation"
    return "Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again."

In [27]:
car_categories_gate('/home/liganium/Documents/Django/Django App- Create & Deploy a Computer Vision Model/images/image1.jpg', cat_list)

Validating that this is a picture of your car...
('n02814533', 'beach_wagon')


'Validation complete - proceed to damage evaluation'